In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import ee
import geemap
import pandas as pd
import os
import json
import geopandas as gpd
import geemap
import time

# Initialize
ee.Authenticate()
ee.Initialize(project='unicef-ccri')

Configuration

In [ ]:
# --- Configuration ---
suffix = 'adm0'
admin_fc = ee.FeatureCollection("projects/unicef-ccri/assets/global_boundary/adm0")
output_folder = f'p1_exposure_{suffix}'
id_name = 'ucode'

In [ ]:

# Load child population and population layers
org_childpop = ee.ImageCollection("projects/unicef-ccri/assets/WorldPop_Con_T_U18")
childpop = org_childpop.mosaic()
scale = org_childpop.first().projection().nominalScale()
totalpop = ee.Image("projects/unicef-ccri/assets/worldpop_1km")
totalpop_res = totalpop.projection().nominalScale()

# Reference image to align CRS and scale
reference_image = ee.Image("projects/unicef-ccri/assets/heatwave_frequency_2014_2023_avg")
target_scale = reference_image.projection().nominalScale()
target_crs = reference_image.projection().crs()

# --- Hazards ---
hazards = [
    {"id": "projects/unicef-ccri/assets/river_flood_r100", "threshold": 0.01, "name": "river_flood_100yr_jrc_2024"},
    {"id": "projects/unicef-ccri/assets/coastal_flood_r100", "threshold": 0, "name": "coastal_flood_100yr_jrc_2024"},
    {"id": "projects/unicef-ccri/assets/storm_giri_rp100", "threshold": 17.5, "name": "tropical_storm_100yr_giri_2024"},
    {"id": "projects/unicef-ccri/assets/ASI_return_level_100yr", "threshold": 30, "name": "agricultural_drought_fao_1984-2023"},
    {"id": "projects/unicef-ccri/assets/drought_spei_copernicus_1940_2024", "threshold": -1.5, "name": "drought_spei_copernicus_1940-2024"},
    {"id": "projects/unicef-ccri/assets/drought_spi_copernicus_1940_2024", "threshold": -1.5, "name": "drought_spi_copernicus_1940-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_frequency_return_level_100yr", "threshold": 'Mean', "name": "heatwave_frequency_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_duration_return_level_100yr", "threshold": 'Mean', "name": "heatwave_duration_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_severity_return_level_100yr", "threshold": 'Mean', "name": "heatwave_severity_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/high_temp_degree_days_return_level_100yr", "threshold": 35, "name": "extreme_heat_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/FIRMS_FRP_90th_percentile", "threshold": 'Mean', "name": "fire_FRP_nasa_2001-2024"},
    {"id": "projects/unicef-ccri/assets/FIRMS_count_90th_percentile", "threshold": 'Mean', "name": "fire_frequency_nasa_2001-2023"},
    {"id": "projects/unicef-ccri/assets/sand_dust_storm_annual", "threshold": 0, "name": "sand_dust_storm_unccd_2024"},
    {"id": "projects/unicef-ccri/assets/pm25_p90_1998_2023", "threshold": 5, "name": "air_pollution_pm25_1998-2023"},
    {"id": "projects/unicef-ccri/assets/Pv_average_2013_2022", "threshold": 0.001, "name": "vectorborne_malariapv_2012-2022"},
    {"id": "projects/unicef-ccri/assets/Pf_average_2013_2022", "threshold": 0.001, "name": "vectorborne_malariapf_2012-2022"}
]

# --- Reference boundary datasets ---
simple_fc = ee.FeatureCollection("projects/unicef-ccri/assets/adm0_simple")
area_fc = ee.FeatureCollection("projects/unicef-ccri/assets/adm0_wfp")
global_geom = simple_fc.geometry()

# --- Compute thresholds where 'Mean' is specified ---
for hazard in hazards:
    if hazard['threshold'] == 'Mean':
        layer = (
            ee.ImageCollection(hazard['id']).mosaic()
            if hazard['name'] in ["river_flood_100yr_jrc_2024", "coastal_flood_100yr_jrc_2024", "tropical_storm_100yr_giri_2024"]
            else ee.Image(hazard['id'])
        )
        th = layer.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=global_geom,
            scale=target_scale,
            bestEffort=True,
            maxPixels=1e13
        ).values().get(0)
        if th:
            hazard['threshold'] = ee.Number(th).getInfo()
        else:
            print(f"⚠️ Skipping {hazard['name']} — no valid global mean.")
            hazard['skip'] = True

# --- Reference areas for thresholds ---
area_singapore = area_fc.filter(ee.Filter.eq('iso3', 'SGP')).first().getNumber('Shape_Area')
area_italy = area_fc.filter(ee.Filter.eq('iso3', 'ITA')).first().getNumber('Shape_Area')
area_sudan = area_fc.filter(ee.Filter.eq('iso3', 'SDN')).first().getNumber('Shape_Area')
area_argentina = area_fc.filter(ee.Filter.eq('iso3', 'ARG')).first().getNumber('Shape_Area')

# --- Get unique admin IDs ---
adm_ids = admin_fc.aggregate_array(id_name).distinct().sort().getInfo()
print(f"Found {len(adm_ids)} unique {id_name} values")

# --- Main Loop ---
for adm_id in adm_ids[0:3]:
    print(f"Processing {adm_id}")
    region_fc = admin_fc.filter(ee.Filter.eq(id_name, adm_id))
    if region_fc.size().getInfo() == 0:
        print(f"⚠️ {adm_id} not found in admin_fc, skipping")
        continue

    region_geom = region_fc.union().geometry()

    # Calculate the area directly from the region's geometry
    # .area() returns the area in square meters.
    # Using a maxError (in meters) is good practice for performance.
    shape_area = region_geom.area(maxError=1000)
    print(shape_area.getinfo())
    # --- Determine simplification tolerance based on thresholds ---
    simplification_tolerance = ee.Number(
        ee.Algorithms.If(
            shape_area.lt(area_singapore),  # smaller than Singapore
            0,  # no simplification
            ee.Algorithms.If(
                shape_area.lt(area_italy),  # smaller than Italy
                1000,  # 1 km simplification
                ee.Algorithms.If(
                    shape_area.lt(area_argentina),  # smaller than Sudan
                    10000,  # 10 km simplification
                    1000000
                )
            )
        )
    )

    # --- Apply simplification only if tolerance > 0 ---
    simplified_geom = ee.Algorithms.If(
        simplification_tolerance.gt(0),
        region_geom.simplify(simplification_tolerance),
        region_geom
    )

    simplified_geom = ee.Geometry(simplified_geom)

    childpop_sum = childpop.reduceRegion(
        ee.Reducer.sum(), simplified_geom, scale=scale, crs='EPSG:4326', maxPixels=1e13).get('b1')
    totalpop_sum = totalpop.reduceRegion(
        ee.Reducer.sum(), simplified_geom, scale=totalpop_res, crs='EPSG:4326', maxPixels=1e13).get('b1')

    ee_features = []
    for hazard in hazards:
        if hazard.get('skip'):
            continue

        name, TH = hazard['name'], hazard['threshold']
        layer = (
            ee.ImageCollection(hazard['id']).mosaic()
            if name in ["river_flood_100yr_jrc_2024", "coastal_flood_100yr_jrc_2024", "tropical_storm_100yr_giri_2024"]
            else ee.Image(hazard['id'])
        )

        if name == "agricultural_drought_fao_1984-2023":
            layer = layer.updateMask(layer.lte(100))
            exposed = childpop.updateMask(layer.gt(ee.Image.constant(TH)))
        elif name in ["drought_spei_copernicus_1940-2024", "drought_spi_copernicus_1940-2024"]:
            exposed = childpop.updateMask(layer.lt(ee.Image.constant(TH)))
        else:
            exposed = childpop.updateMask(layer.gt(ee.Image.constant(TH)))

        hazard_geom = simplified_geom.buffer(5000) if name == "coastal_flood_100yr_jrc_2024" else simplified_geom
        exposed_sum = exposed.reduceRegion(
            ee.Reducer.sum(), hazard_geom, scale=scale, bestEffort=True, crs='EPSG:4326', maxPixels=1e13).get('b1')

        ee_features.append(
            ee.Feature(None, {
                id_name: adm_id,
                'hazard': name,
                'child_population_exposed': exposed_sum,
                'child_population_total': childpop_sum,
                'population_total': totalpop_sum,
                'area': shape_area
            })
        )

    task = ee.batch.Export.table.toDrive(
        collection=ee.FeatureCollection(ee_features),
        description=f"{adm_id}_hazards_{suffix}",
        folder=output_folder,
        fileFormat='CSV',
        selectors=[id_name, 'hazard', 'child_population_exposed', 'child_population_total', 'population_total', 'area']
    )
    task.start()
    # time.sleep(3)

print("✅ All tasks submitted.")


Found 289 unique ucode values
Processing ABW_V2


AttributeError: 'Number' object has no attribute 'getinfo'

Combining into adm0

In [ ]:
import os
import glob
import pandas as pd

# Define the folder and get all CSV file paths
hazard_folder = os.path.join('/content/drive/MyDrive', output_folder)
hazard_files = glob.glob(f'{hazard_folder}/*.csv')

# Read and concatenate all CSVs into one DataFrame
merged_df = pd.concat([pd.read_csv(f) for f in hazard_files], ignore_index=True)

# Output directory
output_dir = '/content/drive/MyDrive/p1_exposure'
os.makedirs(output_dir, exist_ok=True)

# Loop through each unique hazard and export a CSV
for hazard_name in merged_df['hazard'].unique():
    # Filter the data
    df_subset = merged_df[merged_df['hazard'] == hazard_name]

    # Define full path
    output_path = os.path.join(output_dir, f"{hazard_name}_exposure_adm0.csv")

    # Save the CSV
    df_subset.to_csv(output_path, index=False)

    print(f"✅ Saved: {output_path}")


✅ Saved: /content/drive/MyDrive/p1_exposure/river_flood_100yr_jrc_2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/coastal_flood_100yr_jrc_2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/tropical_storm_100yr_giri_2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/agricultural_drought_fao_1984-2023_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/drought_spei_copernicus_1940-2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/drought_spi_copernicus_1940-2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/heatwave_frequency_ecmwf_2014-2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/heatwave_duration_ecmwf_2014-2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/heatwave_severity_ecmwf_2014-2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/extreme_heat_ecmwf_2014-2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/fire_FRP_nasa_2